In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [2]:
import re
import json
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize

In [5]:
# This is the whole dataset of ratebeer
dataset_name = 'whole_dataset'

# Load Data

In [3]:
# Load Data from the original dataset
dir_path = '../Dataset/ratebeer/whole_dataset/split/'
output_dir_path = '../Dataset/ratebeer/whole_dataset/split/'
# Load train dataset
train_review = []
cnt = 0
file_path = os.path.join(dir_path, 'train.txt')
output_file_path = os.path.join(output_dir_path, 'train.json')
with open(file_path) as f:
    with open(output_file_path, 'w') as f_out:
        print("Load file: {}".format(file_path))
        for line in f:
            line_data = json.loads(line)
            item_id = str(line_data[0])     # convert to str type of id
            user_id = str(line_data[1])     # convert to str type of id
            rating = line_data[2]
            review_feat_sent = line_data[3]
            # save the feature word and review text
            review_feat_list = list()
            review_text_list = list()
            for feat_sent in review_feat_sent:
                review_feat_list.append(feat_sent[0])
                review_text_list.append(feat_sent[1])
            review_text = " ".join(review_text_list)
            combined_line_data = {
                'user': user_id,
                'item': item_id,
                'rating': rating,
                'review': review_text
            }
            train_review.append([item_id, user_id, rating, review_text, review_feat_list])
            json.dump(combined_line_data, f_out)
            f_out.write('\n')
            cnt += 1
            if cnt % 100000 == 0:
                print("{} lines loaded.".format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(train_review)))

Load file: ../Dataset/ratebeer/whole_dataset/split/train.txt
100000 lines loaded.
200000 lines loaded.
300000 lines loaded.
400000 lines loaded.
500000 lines loaded.
600000 lines loaded.
700000 lines loaded.
800000 lines loaded.
900000 lines loaded.
1000000 lines loaded.
1100000 lines loaded.
1200000 lines loaded.
1300000 lines loaded.
1400000 lines loaded.
1500000 lines loaded.
1600000 lines loaded.
1700000 lines loaded.
1800000 lines loaded.
1900000 lines loaded.
2000000 lines loaded.
Finish loading train dataset, totally 2015084 lines.


In [4]:
# Load test dataset
test_review = []
cnt = 0
file_path = os.path.join(dir_path, 'test.txt')
output_file_path = os.path.join(output_dir_path, 'test.json')
with open(file_path) as f:
    with open(output_file_path, 'w') as f_out:
        print("Load file: {}".format(file_path))
        for line in f:
            line_data = json.loads(line)
            item_id = str(line_data[0])     # convert to str type of id
            user_id = str(line_data[1])     # convert to str type of id
            rating = line_data[2]
            review_feat_sent = line_data[3]
            # save the feature word and review text
            review_feat_list = list()
            review_text_list = list()
            for feat_sent in review_feat_sent:
                review_feat_list.append(feat_sent[0])
                review_text_list.append(feat_sent[1])
            review_text = " ".join(review_text_list)
            combined_line_data = {
                'user': user_id,
                'item': item_id,
                'rating': rating,
                'review': review_text
            }
            test_review.append([item_id, user_id, rating, review_text, review_feat_list])
            json.dump(combined_line_data, f_out)
            f_out.write('\n')
            cnt += 1
            if cnt % 100000 == 0:
                print("{} lines loaded.".format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(test_review)))

Load file: ../Dataset/ratebeer/whole_dataset/split/test.txt
100000 lines loaded.
200000 lines loaded.
Finish loading train dataset, totally 221194 lines.


# Check Review with 0 sentences

In [7]:
cnt = 0
for train_data_instance in train_review:
    review_text = train_data_instance[3]
    review_sents = sent_tokenize(review_text)
    if len(review_sents) == 0:
        assert review_text == ''
        cnt += 1
print("Number of reviews on train with 0 sentences: {}".format(cnt))

cnt = 0
for test_data_instance in test_review:
    review_text = test_data_instance[3]
    review_sents = sent_tokenize(review_text)
    if len(review_sents) == 0:
        assert review_text == ''
        cnt += 1
print("Number of reviews on test with 0 sentences: {}".format(cnt))

Number of reviews on train with 0 sentences: 0
Number of reviews on test with 0 sentences: 0


# Check Duplicate Reviews

In [8]:
# in train set
trainset_user_item_set = set()
cnt_duplicate_review = 0

for train_data_chunk in train_review:
    item_id = str(train_data_chunk[0])
    user_id = str(train_data_chunk[1])
    if (user_id, item_id) in trainset_user_item_set:
        cnt_duplicate_review += 1
    else:
        trainset_user_item_set.add((user_id, item_id))

print("[Train] Number of duplicate reviews: {}".format(cnt_duplicate_review))
print("[Train] Number of unique reviews: {}".format(len(trainset_user_item_set)))

[Train] Number of duplicate reviews: 43848
[Train] Number of unique reviews: 1971236


In [9]:
# in test set
testset_user_item_set = set()
cnt_duplicate_review = 0

for test_data_chunk in test_review:
    item_id = str(test_data_chunk[0])
    user_id = str(test_data_chunk[1])
    if (user_id, item_id) in testset_user_item_set:
        cnt_duplicate_review += 1
    else:
        testset_user_item_set.add((user_id, item_id))

print("[Test] Number of duplicate reviews: {}".format(cnt_duplicate_review))
print("[Test] Number of unique reviews: {}".format(len(testset_user_item_set)))

[Test] Number of duplicate reviews: 506
[Test] Number of unique reviews: 220688


In [10]:
df_train_data = pd.DataFrame(train_review, columns=['item', 'user', 'rating', 'review', 'features'])
df_test_data = pd.DataFrame(test_review, columns=['item', 'user', 'rating', 'review', 'features'])

In [11]:
df_train_data

,item,user,rating,review,features
0,7121,37,14,the finish is nutty and bready with a note of ...,"[[aftertaste, note, finish, bready], [fizzy, f..."
1,2073,37,15,opaque hue with huge tan creamy head and very ...,"[[hue, lace, tan, head], [coffee, malt, chocol..."
2,2121,37,16,it is balanced with a subdued malt background ...,"[[finish, woody, background, balanced, malt], ..."
3,4662,37,16,light orange hue with good fizzy head and very...,"[[fizzy, orange, hue, lacing, bubbles, light, ..."
4,5415,37,14,the mouth feel is smooth and soft but need a b...,"[[mouth, feel]]"
...,...,...,...,...,...
2015079,3660,6435,14,a decent blond ale . this beer is just good an...,"[[ale], [beer]]"
2015080,16740,6435,16,i love this beer . the smoke taste is present ...,"[[beer], [balanced, taste, smoke], [quality, g..."
2015081,806,6435,19,unique beer with a high abv . this beer is out...,"[[abv, beer], [flavor, beer], [brew], [beer, h..."
2015082,825,6435,19,outstanding sweet beer . this beer is so refre...,"[[beer, sweet], [flavor, beer], [beer]]"


In [13]:
df_test_data

,item,user,rating,review,features
0,1664,37,9,"the aroma has notes of honey , wheat husk , gr...","[[grain, wheat, yeast, esters, aroma, honey, n..."
1,727,37,15,a wonderful combination that allows both hop a...,"[[combination, malt, hop], [finish, top]]"
2,4571,37,12,golden hue with good frothy head and good lace...,"[[frothy, golden, lace, head, hue], [bitternes..."
3,3218,37,14,the taste starts with the expected fruity char...,"[[malts, character, fruity, taste, plenty, sug..."
4,2243,37,14,a play of nutty and smoke qualities over the a...,"[[aftertaste, qualities, smoke], [tad, mouth, ..."
...,...,...,...,...,...
221189,6956,6377,18,pale clear golden color with a thin white head...,"[[color, head, pale, golden], [feel, mouth], [..."
221190,376,6377,14,this started out with a nice aroma of vanilla ...,"[[aroma, chocolate, vanilla], [head, black], [..."
221191,8193,4293,19,very good beer .,[[beer]]
221192,7150,4293,12,really enjoyed this beer,[[beer]]


In [12]:
print("Number of users on train set: {}".format(len(df_train_data['user'].unique())))
print("Number of items on train set: {}".format(len(df_train_data['item'].unique())))
print("Number of users on test set: {}".format(len(df_test_data['user'].unique())))
print("Number of items on test set: {}".format(len(df_test_data['item'].unique())))

Number of users on train set: 6466
Number of items on train set: 19876
Number of users on test set: 6562
Number of items on test set: 19140


## groupby user and item

In [ ]:
groupby_user_item = df_train_data.groupby(['user', 'item'])
cnt_duplicate_review = 0
cnt_user_item_pair = 0
filterd_user_item_pair = []
differ_user_item_pair = []
for key, item in groupby_user_item:
    cur_df_user_item = groupby_user_item.get_group(key)
    cnt_user_item_pair += 1
    if cnt_user_item_pair % 1000 == 0:
        print("{} user-item pairs are being processed.".format(cnt_user_item_pair))
    if len(cur_df_user_item) > 1:
        # if there are multple reviews of this user-item pair, sample 1
        sampled_cur_df_user_item = cur_df_user_item.sample(n=1)
        filterd_user_item_pair.append(sampled_cur_df_user_item)
        rating_list = list(cur_df_user_item['rating'])
        review_text_list = list(cur_df_user_item['review'])
        cnt_duplicate_review += len(cur_df_user_item) - 1
        differ_data_flag = False
        for i in range(len(cur_df_user_item)):
            if rating_list[i] != rating_list[0]:
                differ_data_flag = True
                break
            if review_text_list[i] != review_text_list[0]:
                differ_data_flag = True
                break
        if differ_data_flag:
            differ_user_item_pair.append(cur_df_user_item)
    else:
        filterd_user_item_pair.append(cur_df_user_item)
print("Total unique user-item pair: {}".format(cnt_user_item_pair))
print("Total duplicate reviews: {}".format(cnt_duplicate_review))
print("Total duplicate user-item pair with different reviews: {}".format(
    len(differ_user_item_pair))
)
print("Total saved unique user-item reviews: {}".format(
    len(filterd_user_item_pair))
)